In [1]:
#% This notebook gathers methods from previous iterarions,
#% Make sure that everything you add is properly labeled with its purpose.
#% There are some on-going questions. For example: What are the best practices...
#% ... to properly batch processes multiple files?
#% How do I segregate multiple body parts etc.

In [2]:
#% What do we use to determine the beginning and end of a reach?


In [3]:
import pandas as pd
from preprocessing.classes import FilterandThresholds, simpleKinematics
import os
import re
import numpy as np
from sympy  import *
import matplotlib.pyplot as plt

In [4]:
import re
Multi_indexer = FilterandThresholds('MultiIndex')
files = [file for file in os.listdir('/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/')]


multi_reach= pd.DataFrame()
csvs_animalNum = []



for file in files:
    #read csv

    #We're going to have to avoid hidden files with an if statement
    if not file.startswith('.'):
        csv = pd.read_csv('/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/' + file)


        csv = Multi_indexer.MultiIndexMaker(csv)

        #We also have to add a column for reach count - otherwise files from the same animal will connect with one an other.
        #We can use regex again to look for _int_ pattern in the file name, and write in a new column.


        #Search for animal number from the file name and put it in a new column
        csvs_animalNum = list([int(n) for n in re.findall(r'\d+', file) if len(n) == 5 ] * len(csv.index))

        #Search for animal reach number

        csv['Animal Infor', 'Reach Number'] = [re.findall(r'_\d{1,2}_', str(file)).pop(0).lstrip('_').rstrip('_')] * len(csv.index)
        #print(strikeString)

        #We're going to add a cumsum and and a diffeference column to each strike per animal. We're going to align by y values

        #look for animal condition
        animalCond_blindBool = re.search(r'_blind_', str(file))
        animalCond_contBool = re.search(r'_control_', str(file))
        if animalCond_blindBool:
            csv['Animal Infor', 'Condition'] = list(['blind'] * len(csv.index))

        elif animalCond_contBool:
            csv['Animal Infor', 'Condition'] = list(['control'] * len(csv.index))



        csv['Animal Infor','ID'] = csvs_animalNum

        multi_reach = pd.concat([multi_reach, csv], ignore_index = True)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/'

In [ ]:
# multi_reach.to_csv('/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/tocsv.csv')

In [ ]:
# import re
# import matplotlib.pyplot as plt
#
# Multi_indexer = FilterandThresholds('MultiIndex')
# files = [file for file in os.listdir('/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/')]
# file = files[1]
# file
# multi_reach= pd.DataFrame()
# csvs_animalNum = []
# if not file.startswith('.'):
#     csv = pd.read_csv('/Users/carlospineda/Desktop/TEMP Reaching Results for Code Dev/camera2/' + file)
#
#
#     csv = Multi_indexer.MultiIndexMaker(csv)
#
#     #We also have to add a column for reach count - otherwise files from the same animal will connect with one an other.
#     #We can use regex again to look for _int_ pattern in the file name, and write in a new column.
#
#
#     #Search for animal number from the file name and put it in a new column
#     csvs_animalNum = list([int(n) for n in re.findall(r'\d+', file) if len(n) == 5 ] * len(csv.index))
#
#     #Search for animal reach number
#
#     csv['Animal Infor', 'Reach Number'] = [re.findall(r'_\d{1,2}_', str(file)).pop(0).lstrip('_').rstrip('_')] * len(csv.index)
#     #print(strikeString)
#
#     #We're going to add a cumsum and and a diffeference column to each strike per animal. We're going to align by y values
#
#     #look for animal condition
#     animalCond_blindBool = re.search(r'_blind_', str(file))
#     animalCond_contBool = re.search(r'_control_', str(file))
#     if animalCond_blindBool:
#         csv['Animal Infor', 'Condition'] = list(['blind'] * len(csv.index))
#
#     elif animalCond_contBool:
#         csv['Animal Infor', 'Condition'] = list(['control'] * len(csv.index))
#
#
#
#     csv['Animal Infor','ID'] = csvs_animalNum
#
#     multi_reach = pd.concat([multi_reach, csv], ignore_index = True)
# multi_reach.head()

#

In [ ]:
#We should add the ability to filter predictions- in this case accurate tracking isnt imperative.


bodypartSelector = FilterandThresholds('TestingThis')

Platform = bodypartSelector.bodypart_selector(multi_reach, 'Platform', likelihoodSelect=True)
Platform.head()

In [ ]:

#This looks kind of fucked, and not cure like. Also, there are no labels.
#The data is also not mapping - but this is probably due to differencs in the analysis.

import seaborn as sns

WristR = bodypartSelector.bodypart_selector(multi_reach, 'WristR', likelihoodSelect=True)


plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time', y = 'y', hue = 'Condition', data = WristR)
plt.xlabel('Frame', fontsize=20)
plt.ylabel('RFRy_disp (pixels)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
#ax.invert_yaxis()
plt.legend(loc=1, prop={'size': 19})


In [ ]:
#Try one
WristL = bodypartSelector.bodypart_selector(multi_reach, 'WristL' , likelihoodSelect=True)

#Group dataframe by reach and number
WristLbyReach = WristL.groupby(['Reach Number', 'ID'])
keys = WristLbyReach.groups.keys()
x =[]
reaches = []
for key in keys:
    reach = WristLbyReach['y'].get_group(key)
    Time =  WristLbyReach.get_group(key)
    reaches.append(reach)
    x.append(Time)
    #print(reaches)

In [ ]:
#Now we're iterating throught he groups.
constantAccVelocity = pd.DataFrame()
WristLExtendCleaned = pd.DataFrame()
cleanedData = pd.DataFrame()
velocityChancla = simpleKinematics('tester')
curvePeaks = simpleKinematics('multi_reach')
splitter = FilterandThresholds('Tester single reach')

for reach, ID in WristLbyReach:
    ID = pd.DataFrame(ID)
    #Calculate the raw velocity and store in dataframe -
    velocity = velocityChancla.velocity(ID, 'y')

    trough_output_velocity, peak_output_velocity, min_indy, velocity, promineneces_velocity = curvePeaks.curveExtractor(velocity, trough_height=2, trough_width=None, peak_height=2000, peak_width=None, prominences=45, gausFilter=False)

    #We need to concatenate all signals
    Cleaned, ExtendCleaned = splitter.splitter(signal=ID, velocity_peaks=peak_output_velocity, whichPlane='y' )
    Cleaned['Time_Aligned'] = np.arange(0, (0.003 * len(Cleaned)), 0.003)
    ExtendCleaned['Time_Aligned'] = np.arange(0, (0.003 * len(ExtendCleaned)), 0.003)

    cleanedData = pd.concat([cleanedData, Cleaned], ignore_index=True)
    WristLExtendCleaned = pd.concat([WristLExtendCleaned, ExtendCleaned], ignore_index=True)

In [ ]:
ReachNumClean = cleanedData[cleanedData['Reach Number'] == '20']
ReachNum = cleanedData[cleanedData['Reach Number'] == '20']
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time_Aligned', y = 'y', hue = 'Condition', data = ReachNumClean)
plt.xlabel('Frame', fontsize=20)
plt.ylabel('RFRy_disp (pixels)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
plt.legend(loc=1, prop={'size': 19})

In [ ]:
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time_Aligned', y = 'y', hue = 'Condition', data = WristLExtendCleaned)
plt.xlabel('Frame', fontsize=20)
plt.ylabel('RFRy_disp (pixels)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
plt.legend(loc=1, prop={'size': 19})

In [ ]:
# WristLExtendCleaned = WristLExtendCleaned.groupby('Reach Number')
# size = WristLExtendCleaned.size()
# print(size)

In [ ]:
#Using np.gradient to calculate velocty
x_t = np.gradient(WristLExtendCleaned['Time'])
y_t = np.gradient(WristLExtendCleaned['y'])

velocity = np.array([ [x_t[i], y_t[i]] for i in range(x_t.size)])
#print(velocity)
x_a = np.gradient(x_t)
y_a = np.gradient(y_t)

acceleraton = np.array([ [x_a[i], y_a[i]] for i in range(x_a.size)])
#print(acceleraton)
plt.plot(velocity)
plt.plot(acceleraton)


plt.show()

In [ ]:
#Now calculate speed
speed = np.sqrt(x_t * x_t + y_t * y_t)
print(speed)

plt.plot(speed)


plt.show()

#This finds the max peak - we need to create a window around the pea
#How do we use this number to further cut the dataframe?
#Lets try bounding the dataframe by whatever peaks there are- this should be robust in case there are higher velocity peaks
#We can look for the nearest peak behind the max index.

#Lets create an algorithm to find points in the velocity frame that exeeds 20% of the peak velocit

In [ ]:
# from scipy.optimize import curve_fit
# def objective(x, a, b, c, d, e, f):
#     return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + f
#
# x, y = WristLExtendCleaned['Time'], WristLExtendCleaned['y']
#
# popt, _ = curve_fit(objective, x, y)
#
# a, b, c, d, e, f = popt
# print('y = %.5f * x + %.5f * x^2 + %.5f + x^3 + %.5f + x^4 + %.5f + x^5 + %.5f' % (a, b, c, d, e, f))
#
# plt.scatter(x, y)
#
#
# x_line = np.linspace(min(x), max(x))
# y_line = objective(x_line, a, b, c, d, e, f)
#
# plt.plot(x_line, y_line, '--', color='red')
# plt.gca().invert_yaxis()
# plt.show()
#

In [ ]:
# x = Symbol('x')
# f = (1312414 * x) + (-4891866 * x**2) + (9013961* x**3) + (-8221780 * x**4) + (2972624 * x**5) + -138898
# f_prime = f.diff(x)
# print(f_prime)

In [ ]:
Snout = bodypartSelector.bodypart_selector(multi_reach, 'Snout' , likelihoodSelect=True)
WristL.head()

plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time', y = 'y', hue = 'Condition', data = Snout)
plt.xlabel('Frame', fontsize=20)
plt.ylabel('Snout_disp (cm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)

ax.invert_yaxis()

plt.legend(loc=1, prop={'size': 19})


In [ ]:

# fig, ax = plt.subplots(figsize=(8,6))
#
# #Initial test is okay
# WristR.groupby(['Condition', 'Reach Number']).plot(kind = 'line', x = 'Time', y = 'x', ax = ax)
# ax.invert_yaxis()
# plt.show()




# axs[0].grid()
# axs[1].grid()
# axs[2].grid()
# fig.suptitle('X-Displacement and Y-Displacement Reach of Control')
#
# plt.xlabel('Time (s)')
# plt.setp(axs[0], ylabel='x_displacement (pix)')
# plt.setp(axs[1], ylabel='y_displacement (pix)')
# plt.setp(axs[2], ylabel = 'likelihood (0.1)')
#
# multi_reach.Wrist
# #Plot different body parts from the csv
# axs[0].plot(csv.coords, csv.x, color = 'r')
# axs[1].plot(multi_reach_filtered.coords, multi_reach_filtered.y, color = 'r')
# axs[2].plot(multi_reach_filtered.coords, multi_reach_filtered.likelihood, color = 'b')
# #Add Z component of whatever body part here.
# #We need to cushion the filtered dataset so that the time scale is not distorted - try interpollating
# plt.gca().invert_yaxis()
# plt.show()
